In [8]:
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [9]:
V_max = 4.2;        V_min = 2.5;
exp  = pybamm.Experiment(
    [ (
        f"Discharge at 1 C until {V_min} V", 
        f"Charge at 1 C until {V_max} V", 
        f"Hold at {V_max} V until C/5")    ] * 1 )  
Model_0  = pybamm.lithium_ion.DFN(options={
    "solvent diffusion": "single no consume wo refill",
    "electrolyte conductivity": "full"  ,
    }) 
Model_1  = pybamm.lithium_ion.DFN(options={
    "solvent diffusion": "double spatial consume w refill",
    "electrolyte conductivity": "sol full"  ,
    }) 

In [10]:
def EC_diffusivity_5E_5(c_e, c_EC , T):
    D_ec_dim = (
        (c_EC >= 0 ) * 5e-5 
        +  (c_EC < 0 ) * 0 
    )
    return D_ec_dim
Para_0=pybamm.ParameterValues("Li2023_ECdrag")
Para_0.update({'Negative electrode porosity':0.10})
Para_0.update({"EC diffusivity in electrolyte [m2.s-1]":5e-10})
Para_0.update({"EC Lithium ion cross diffusivity [m2.s-1]":1.5e-12})

Para_1=pybamm.ParameterValues("Li2023_ECdrag")
Para_1.update({'Negative electrode porosity':0.10})
Para_1.update({"EC diffusivity in electrolyte [m2.s-1]":5e-10})
Para_1.update({"EC Lithium ion cross diffusivity [m2.s-1]":1.5e-12})

In [14]:
Para_0["Electrolyte conductivity [S.m-1]"]

<function pybamm.input.parameters.lithium_ion.Li2023_ECdrag.electrolyte_conductivity_Andrew2022(x, y, T)>

In [11]:
Sim_0    = pybamm.Simulation(
    Model_0, experiment = exp,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)       
Sol_0    = Sim_0.solve()

In [12]:
Sim_1    = pybamm.Simulation(
    Model_1, experiment = exp,
    parameter_values = Para_1,
    solver = pybamm.CasadiSolver(),)       
Sol_1    = Sim_1.solve()

In [13]:
label = ["Normal DFN","Double transport + Sol affect potential",] 
output_variables3 = [
    "Electrolyte concentration [mol.m-3]",
    "EC concentration [mol.m-3]", 
    "Terminal voltage [V]", 
    "Electrolyte potential [V]",
]
quick_plot = pybamm.QuickPlot(
    [Sol_0,Sol_1], output_variables3,label,variable_limits='tight',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.8273944574653926, step=0.01827394457465392…